In [123]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import normalize

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

import time

tf.keras.backend.clear_session()  # For easy reset of notebook state.

2.1.0


In [124]:
x = np.loadtxt('coef_mat_two_photon.dat')
#x = np.loadtxt('coef_mat_one_photon.dat')
#y = np.loadtxt('coef_mat_zero_photon.dat')
y = np.loadtxt('ionization_probability.dat')
#x = y[:,2]
#print(ion.shape)

#plt.plot(ion[:,2],'o')
print(x.shape)
print(y.shape)
#print(y[0:1])
# standardize dataset
x = StandardScaler().fit_transform(x)
#y = StandardScaler().fit_transform(y)
#x = StandardScaler().fit_transform(y.reshape(len(y),3))[:,1]
y = StandardScaler().fit_transform(y.reshape(len(y),3))[:,2]
print("")
print(x.shape)
print(y.shape)
print(len(y))

(100001, 40)
(100001, 3)

(100001, 40)
(100001,)
100001


In [125]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.05)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.05263)

print("\nx:")
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print("y:")
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

#y_test_inv = StandardScaler().fit(y_test.reshape(len(y_test),40)).inverse_transform(y_test)
#y_test_inv.shape
#plt.plot(y_test_inv)
print("y_len = ",len(y_train))

print(x_train[0].shape[0])
print(x_train.shape[0])


x:
(90000, 40)
(5001, 40)
(5000, 40)
y:
(90000,)
(5001,)
(5000,)
y_len =  90000
40
90000


In [126]:
x_train = x_train.reshape(x_train.shape[0], 40, 1, 1)
y_train = y_train.reshape(y_train.shape[0], 1, 1, 1)

x_val = x_val.reshape(x_val.shape[0], 40, 1, 1)
y_val = y_val.reshape(y_val.shape[0], 1, 1, 1)

print("\nx:")
print(x_train.shape)
print(x_val.shape)


print("y:")
print(y_train.shape)
print(y_val.shape)


print(x_train.shape[1:])

model = Sequential()
model.add(Conv2D(64, 3, 3, padding='same', input_shape=x_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(128, 3, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Conv2D(264, 3, 3, activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
model.summary()

model.compile(loss='mean_squared_error', optimizer='adam')



x:
(90000, 40, 1, 1)
(5000, 40, 1, 1)
y:
(90000, 1, 1, 1)
(5000, 1, 1, 1)
(40, 1, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 1, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 1, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 1, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 1, 128)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 264)         304392    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 264)         0         
____________________________________

In [127]:
history = model.fit(x_train, y_train,
          epochs=3,
          batch_size=16,
          validation_data=(x_val, y_val))

Train on 90000 samples, validate on 5000 samples
Epoch 1/3
90000/90000 [==============================] - 15s 169us/sample - loss: 1.0009 - val_loss: 0.9812
Epoch 2/3
90000/90000 [==============================] - 16s 178us/sample - loss: 0.9998 - val_loss: 0.9777
Epoch 3/3
90000/90000 [==============================] - 16s 172us/sample - loss: 0.9997 - val_loss: 0.9783


In [128]:
# evaluate the model
train_mse = model.evaluate(x_train, y_train, verbose=0)
test_mse = model.evaluate(x_test, y_test, verbose=0)
print('Train:', train_mse,' Test: ',test_mse)
# plot loss during training
plt.title('Loss / Mean Squared Error')
plt.plot(history.history['loss'], label='loss(train)')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.savefig('loss.pdf',formate='pdf')
plt.show()

ValueError: Error when checking input: expected conv2d_input to have 4 dimensions, but got array with shape (5001, 40)

In [ ]:
#prediction
y_train_predict = model.predict(x_train, batch_size = 1)
y_test_predict = model.predict(x_test, batch_size = 1)